In [66]:
import numpy
from einsteinpy.symbolic import EinsteinTensor, MetricTensor
from sympy import *

# Quasinormal modes for electrically charged black brane in AdS5

#### TODO? Copy introduction

### Fluctuations for $k_T$ = 0 and $k_L = k$

#### Riemannian geometry code:

#### Metric ansatz:
Use Poincaré patch coordinates.

In [67]:
# TODO: What are these
def ZeroV(v):
    return 0

def Zero(v, r):
    return 0

# Define all necessary variables
l, m = symbols('l, m')
g, q, r = symbols('g, q, r', positive=True)
mu = symbols('\mu')
t, x, y, z = symbols('t, x, y, z')
coords = {t, x, y, z , r}
coords_str = ['t', 'x', 'y', 'z', 'r']

# TODO: What are these
Q = sqrt(3 * g**2 * q**2 / 4)
A_t = -Q / r**2 + mu
# CHECK: Q^2=3 g^2/(2 \[Kappa]^2 l^2) q^2

Define the RN black brane background metric:  
TODO: Compare with paper

In [68]:
f = 1 - m * l**2 / r**4 + q**2 * l**2 / r**6

# RN metric components
g_tt = -r**2 / l**2 * f
g_xx = r**2 / l**2 
g_yy = r**2 / l**2
g_zz = r**2 / l**2
g_rr = l**2 / (r**2 * f)

# Construct RN metric
ansatz = diag(g_tt, g_xx, g_yy, g_zz, g_rr) 
display(ansatz)

Matrix([
[-r**2*(-l**2*m/r**4 + l**2*q**2/r**6 + 1)/l**2,         0,         0,         0,                                               0],
[                                             0, r**2/l**2,         0,         0,                                               0],
[                                             0,         0, r**2/l**2,         0,                                               0],
[                                             0,         0,         0, r**2/l**2,                                               0],
[                                             0,         0,         0,         0, l**2/(r**2*(-l**2*m/r**4 + l**2*q**2/r**6 + 1))]])

These transformations show that the chosen metric is equivalent to Starinet's metric?

In [69]:
# Compare to Starinets' metric:
# Simplify the metric through replacements

# Replace q -> 0, l -> 1, m -> 1 and simplify
# replacements = [(q, 0), (l, 1), (m, 1)]
# ansatz = ansatz.subs(replacements)
# display(ansatz)

# Introduce new variable u as r = 1/sqrt(u)
# u = symbols('u')
# ansatz = simplify(ansatz.subs(r, 1/sqrt(u)))
# display(ansatz)

# Take the bottom right element and multiply by 1/(4 u**3)
# a = simplify(ansatz[4,4] * 1/(4*u**3))
# display(a)

##### Choose radial gauge and impose symmetry of the metric:
Define 5x5 matrix with components h_ii  
We already put in some assumptions in:  
1. The variables only depend on (t,z,r) not on (t,x,y,z,r)
2. h_ir = 0
3. The matrix is self-adjoint

In [70]:
h_functions = [[Function(f'h_{var2}{var1}')(t, z, r) for var1 in coords_str] for var2 in coords_str]
fluctuations = Matrix(h_functions)

# 2nd step: h_ir = 0
for i in range(5):
    fluctuations[i, 4] = 0

# Change metric components to make self-adjoint
for i in range(1, 5):
    for j in range(i):
        fluctuations[i, j] = fluctuations[j, i]

display(fluctuations)


Matrix([
[h_tt(t, z, r), h_tx(t, z, r), h_ty(t, z, r), h_tz(t, z, r), 0],
[h_tx(t, z, r), h_xx(t, z, r), h_xy(t, z, r), h_xz(t, z, r), 0],
[h_ty(t, z, r), h_xy(t, z, r), h_yy(t, z, r), h_yz(t, z, r), 0],
[h_tz(t, z, r), h_xz(t, z, r), h_yz(t, z, r), h_zz(t, z, r), 0],
[            0,             0,             0,             0, 0]])

We also create a fluctuations vector, which also has no radial component

In [71]:
a_functions = [Function(f'a_{var}')(t, z, r) for var in coords_str]
fluctuationsA = Matrix(a_functions)

# No radial component
fluctuationsA[4] = 0

display(fluctuationsA)

Matrix([
[a_t(t, z, r)],
[a_x(t, z, r)],
[a_y(t, z, r)],
[a_z(t, z, r)],
[           0]])

##### Define the metric and gauge field including fluctuations and compute the GR tensors:

In [74]:
eps = symbols('eps')
ll = 1

GG = ansatz + eps * fluctuations + Matrix(5, 5, [O(eps**2)]*5*5)
AA = A_t * Matrix([1,0,0,0,0]) + eps * fluctuationsA + Matrix(5, 1, [O(eps**2)]*5)

M_list = [[GG[i, j] for j in range(GG.shape[1])] for i in range(GG.shape[0])]
GG = MetricTensor(M_list, syms=[t, x, y, z, r])

display(GG)
#use the metric to calculate the Einstein tensor
einst = EinsteinTensor.from_metric(GG)
display(einst.tensor())

# This step uses a mathematica packages ://
# https://library.wolfram.com/infocenter/MathSource/4484/
# Alternatively use einsteinpy?

# syms = symbols("t chi theta phi")
# t, ch, th, ph = syms
# m = diag(-1, cos(t) ** 2, cos(t) ** 2 * sinh(ch) ** 2, cos(t) ** 2 * sinh(ch) ** 2 * sin(th) ** 2).tolist()
# metric = MetricTensor(m, syms)
# einst = EinsteinTensor.from_metric(metric)
# display(einst.tensor())


MetricTensor[[-r**2*(-l**2*m/r**4 + l**2*q**2/r**6 + 1)/l**2 + eps*h_tt(t, z, r) + O(eps**2), eps*h_tx(t, z, r) + O(eps**2), eps*h_ty(t, z, r) + O(eps**2), eps*h_tz(t, z, r) + O(eps**2), O(eps**2)], [eps*h_tx(t, z, r) + O(eps**2), r**2/l**2 + eps*h_xx(t, z, r) + O(eps**2), eps*h_xy(t, z, r) + O(eps**2), eps*h_xz(t, z, r) + O(eps**2), O(eps**2)], [eps*h_ty(t, z, r) + O(eps**2), eps*h_xy(t, z, r) + O(eps**2), r**2/l**2 + eps*h_yy(t, z, r) + O(eps**2), eps*h_yz(t, z, r) + O(eps**2), O(eps**2)], [eps*h_tz(t, z, r) + O(eps**2), eps*h_xz(t, z, r) + O(eps**2), eps*h_yz(t, z, r) + O(eps**2), r**2/l**2 + eps*h_zz(t, z, r) + O(eps**2), O(eps**2)], [O(eps**2), O(eps**2), O(eps**2), O(eps**2), l**2/(r**2*(-l**2*m/r**4 + l**2*q**2/r**6 + 1)) + O(eps**2)]]

NotImplementedError: MRV set computation for functions in several variables not implemented.

In [ ]:
quit()

: 

In [ ]:
from sympy import symbols, Function, diag, Matrix
from einsteinpy.symbolic import MetricTensor, EinsteinTensor

# define your symbols and matrices
eps = symbols('eps')
t, z, r = symbols('t z r')
coords_str = ['t', 'x', 'y', 'z', 'r']
g_tt, g_xx, g_yy, g_zz, g_rr = symbols('g_tt g_xx g_yy g_zz g_rr')  # define these symbols

h_functions = [[Function(f'h_{var2}{var1}')(t, z, r) for var1 in coords_str] for var2 in coords_str]
fluctuations = Matrix(h_functions)

# 2nd step: h_ir = 0
for i in range(5):
    fluctuations[i, 4] = 0

# Change metric components to make self-adjoint
for i in range(1, 5):
    for j in range(i):
        fluctuations[i, j] = fluctuations[j, i]

ansatz = diag(g_tt, g_xx, g_yy, g_zz, g_rr)

GG = ansatz + eps * fluctuations

# Create a metric tensor
metric = MetricTensor(GG.tolist(), (t,x,y, z, r))

# Calculate the Einstein tensor
einst = EinsteinTensor.from_metric(metric)

display(einst.tensor())